In [81]:
import scipy.stats as st
import pylab
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("example") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memoryOverhead", "1g") \
    .config("spark.sql.hive.convertMetastoreOrc", "false") \
    .enableHiveSupport() \
    .getOrCreate()


import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import when ,concat,lit , round, mean ,median , col, lit, concat_ws
from pyspark.sql import functions as F
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

23/10/05 22:54:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
df = spark.read.csv("/Users/nithinkumar/Downloads/df1.csv",header=True,inferSchema=True)

23/10/05 21:15:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Data Preprocessing

In [3]:
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

Remaining columns: 133


Number of rows: 2125537


In [4]:
columns_to_drop=['changed_x','changer_x','creator_x','pkey_x'
                 ,'pkey_y','changed_y','deleted_y','deleted_x',
                 'version_x','version_y','changer_y','created_x',
                 'creator_y','trans_status_y','acquirer_country_code_ext',
                 'acs_interface','acs_operatorid','acs_reference_number',
                 'acs_signed_content','authentication_type','message_version_y',
                 'instrument','authentication_value','ds_reference_number',
                 'ds_transid','eci','entity_id','error_code_x',
     'error_component_x','error_description_x','error_detail_x',
                 'error_message_type_x','error_msg_posted_x','kit_no',
                 'merchant_fraud_rate','message_version_x','purchase_amount',
                 'purchase_exponent','sca_exemption_ind','last_msg_received',
                 'sdk_transid_x','secure_corporate_payment','threedsrequestorid',
                 'threedsserver_transid_x','threeriind','trans_status_reason',
                 'masked_pan','partner_name','product_code','response_code_x','bank_code_x',
     'hashed_card','ruleexecutedid','msgreason','rule_result','attempts',
                 'authvalue','challenge_no_entry_counter','error_component_x'
                 ,'error_code_y','error_description_y','error_detail_y','error_message_type_y',
                 'error_msg_posted_y','message_type_y','message_version_y','otp','sdk_transid_y',
                 'threedssession_data','otpreferenceid','bank_code_y','response_code_y','opt_method',
                 'bank_code_y','response_code_y','masked_pan','sdk_transid_y','threedsserver_transid_y',
     'deleted_y','error_code_y','bin','acs_counter_atos','sdk_counter_stoa',
                 'threedsrequestor_challenge_ind','acs_counter_atos','interaction_counter',
                 'key_index','threedsserver_ref_number','acsurl','auth_type','auth_method',
                 'network','acs_challenge_mandated','message_type_y','challenge_received',
                 'acs_counter_stoa','dsurl','acs_ui_template','sdk_reference_number','riskscore',
                 'ruledetails','frm_tran_id','riskstatus_code',
     'riskdescription','acs_ui_type','error_component_y','error_description_y',
                 'error_detail_y','error_message_type_y','error_msg_posted_y',
                 'switch_reason','flow','cardholder_name','cek','oob_status','oob_description',
                 'second_auth','message_type_x']
df=df.drop(*columns_to_drop)

In [5]:
#checking the columns of the dataset
print(df.columns)

['acs_transid', 'challenge_completion_ind', 'challenge_window_size', 'msg_reason', 'otp_triggers', 'otp_attempts', 'trans_status_x', 'created_y', 'acct_number', 'acquirerbin', 'acquirer_merchantid', 'card_expiry_date', 'device_channel', 'merchant_country_code', 'merchant_name', 'message_category', 'notificationurl', 'pay_token_ind', 'purchase_currency', 'purchase_date', 'threedscomp_ind', 'threedsrequestor_authentication_ind', 'threedsrequestor_name', 'threedsrequestorurl', 'threedsserverurl', 'trans_type', 'browser_user_agent', 'browserip', 'amount']


In [6]:
#null values count
from pyspark.sql.functions import col

# Assuming df is your DataFrame
null_counts = []

# Iterate over all columns in the DataFrame
for col_name in df.columns:
    # Count the number of null values for each column
    null_count = df.where(col(col_name).isNull()).count()
    # Append the result to the list
    null_counts.append((col_name, null_count))

# Display the null counts for each column
for col_name, count in null_counts:
    print(f"Column : '{col_name}' has {count} null values.")


Column : 'acs_transid' has 6337 null values.
Column : 'challenge_completion_ind' has 118371 null values.
Column : 'challenge_window_size' has 51518 null values.
Column : 'msg_reason' has 121470 null values.
Column : 'otp_triggers' has 1777006 null values.
Column : 'otp_attempts' has 2078023 null values.
Column : 'trans_status_x' has 238242 null values.
Column : 'created_y' has 680 null values.
Column : 'acct_number' has 6047 null values.
Column : 'acquirerbin' has 6325 null values.
Column : 'acquirer_merchantid' has 3678 null values.
Column : 'card_expiry_date' has 8897 null values.
Column : 'device_channel' has 12447 null values.
Column : 'merchant_country_code' has 15289 null values.
Column : 'merchant_name' has 15277 null values.
Column : 'message_category' has 12462 null values.
Column : 'notificationurl' has 56909 null values.
Column : 'pay_token_ind' has 1503677 null values.
Column : 'purchase_currency' has 12507 null values.
Column : 'purchase_date' has 12490 null values.
Column

In [7]:
#dulicate values
df=df.dropDuplicates()
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

Remaining columns: 29


23/10/05 21:19:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of rows: 2124578


In [8]:
df.printSchema()

root
 |-- acs_transid: string (nullable = true)
 |-- challenge_completion_ind: string (nullable = true)
 |-- challenge_window_size: string (nullable = true)
 |-- msg_reason: string (nullable = true)
 |-- otp_triggers: string (nullable = true)
 |-- otp_attempts: double (nullable = true)
 |-- trans_status_x: string (nullable = true)
 |-- created_y: string (nullable = true)
 |-- acct_number: string (nullable = true)
 |-- acquirerbin: string (nullable = true)
 |-- acquirer_merchantid: string (nullable = true)
 |-- card_expiry_date: double (nullable = true)
 |-- device_channel: string (nullable = true)
 |-- merchant_country_code: double (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- message_category: string (nullable = true)
 |-- notificationurl: string (nullable = true)
 |-- pay_token_ind: string (nullable = true)
 |-- purchase_currency: double (nullable = true)
 |-- purchase_date: double (nullable = true)
 |-- threedscomp_ind: string (nullable = true)
 |-- threedsrequ

In [9]:
df = df.select([col(column).alias(column.strip()) for column in df.columns])

In [10]:
df=df.filter(col('created_y')!='ACS_SYSTEM')

In [11]:
#dulicate values
df=df.dropDuplicates()
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

Remaining columns: 29


Number of rows: 2121733


In [12]:
df=df.filter(col('created_y')!='10021.0')

In [13]:
#dulicate values
df=df.dropDuplicates()
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

Remaining columns: 29


Number of rows: 2116076


In [14]:
from pyspark.sql.functions import to_date,date_format

df = df.withColumn("created", to_date("created_y", "yyyy-MM-dd"))
df = df.withColumn("created_time", date_format("created_y", "HH"))

#d = df.withColumn("created", col("created_y").substr(12, 8))


In [15]:
from pyspark.sql.functions import when, col


result_df = df.withColumn("otp_triggers",
    when(col("otp_triggers").isNull(), 1).otherwise(col("otp_triggers") + 1)
)
result_df = result_df.fillna({"otp_triggers": 1})
print("Remaining columns:", len(result_df.columns))
print("Number of rows:", result_df.count())

Remaining columns: 31


Number of rows: 2116076


In [17]:
from pyspark.sql import functions as f
for col in result_df.columns:
    print(col,result_df.select(col).distinct().count())

acs_transid 2116038


challenge_completion_ind 3


challenge_window_size 6


msg_reason 30


otp_triggers 24


...

browser_user_agent 147179


browserip 659775


amount 158377


created 243


created_time 24


In [18]:
#null values count
from pyspark.sql.functions import col

# Assuming df is your DataFrame
null_counts = []
rdf=result_df.count()

# Iterate over all columns in the DataFrame
for col_name in result_df.columns:
    # Count the number of null values for each column
    null_count = result_df.where(col(col_name).isNull()).count()
    Null_percentage=null_count*100/rdf
    print(col_name,rdf,round(Null_percentage))
    if Null_percentage>30:
        result_df=result_df.drop(col_name)

acs_transid 2116076 0.0


challenge_completion_ind 2116076 5.5357652560683075


challenge_window_size 2116076 2.403032783321582


msg_reason 2116076 5.69535309695871
otp_triggers 2116076 0.0


otp_attempts 2116076 98.02195195257637


...

browser_user_agent 2116076 2.4064353076165506


browserip 2116076 6.185694653689187


amount 2116076 0.303628036044074


created 2116076 0.0


created_time 2116076 0.0


In [19]:
result_df.printSchema()

root
 |-- acs_transid: string (nullable = true)
 |-- challenge_completion_ind: string (nullable = true)
 |-- challenge_window_size: string (nullable = true)
 |-- msg_reason: string (nullable = true)
 |-- otp_triggers: double (nullable = false)
 |-- trans_status_x: string (nullable = true)
 |-- created_y: string (nullable = true)
 |-- acct_number: string (nullable = true)
 |-- acquirerbin: string (nullable = true)
 |-- acquirer_merchantid: string (nullable = true)
 |-- card_expiry_date: double (nullable = true)
 |-- device_channel: string (nullable = true)
 |-- merchant_country_code: double (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- message_category: string (nullable = true)
 |-- notificationurl: string (nullable = true)
 |-- purchase_currency: double (nullable = true)
 |-- purchase_date: double (nullable = true)
 |-- threedscomp_ind: string (nullable = true)
 |-- threedsrequestor_authentication_ind: string (nullable = true)
 |-- threedsrequestor_name: string (n

In [20]:
result_df.createOrReplaceGlobalTempView("data_preprocessing")

In [21]:
spark.sql("""SELECT created,count(*) FROM global_temp.data_preprocessing
          group by created
          order by  2""").show(5)

23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 21:47:40 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------+
|   created|count(1)|
+----------+--------+
|2023-01-01|    1071|
|2023-01-29|    1421|
|2023-01-15|    1460|
|2023-01-02|    1543|
|2023-01-22|    1546|
+----------+--------+
only showing top 5 rows



In [22]:
%time
spark.sql("""SELECT purchase_currency  FROM global_temp.data_preprocessing
          group by 1""").show(10)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 39.1 µs


+-----------------+
|purchase_currency|
+-----------------+
|            170.0|
|            608.0|
|            702.0|
|            818.0|
|              0.0|
|            524.0|
|            840.0|
|            936.0|
|            124.0|
|            348.0|
+-----------------+
only showing top 10 rows



In [25]:
%time
spark.sql("""SELECT count(distinct purchase_currency)  FROM global_temp.data_preprocessing
          """).show(10)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


+---------------------------------+
|count(DISTINCT purchase_currency)|
+---------------------------------+
|                              178|
+---------------------------------+



result_df.printSchema()

# Metabase

In [ ]:
NOD=180

Trans_status_Percentage

In [68]:
NOD=180
%time
output=spark.sql(f"""SELECT  trans_status_x as trans_status , round(count(*)*100/
(SELECT  count(*) as Total_Count
     FROM global_temp.data_preprocessing
    where created>=date_sub(current_date(),{NOD})),2) total_count_percentage
    ,count(*) as total_count
     FROM global_temp.data_preprocessing
    where created>=date_sub(current_date(),{NOD})
    group by 1
          """)
output.show(10)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 8.34 µs


23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:33:00 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+----------------------+-----------+
|trans_status|total_count_percentage|total_count|
+------------+----------------------+-----------+
|        null|                 10.84|     172692|
|           Y|                  84.5|    1345931|
|           U|                  0.17|       2680|
|           N|                  4.49|      71511|
+------------+----------------------+-----------+



In [83]:
NOD=180
%time
output=spark.sql(f"""
SELECT  count(*) as Total_Amount, msg_reason as trans_status
     FROM global_temp.data_preprocessing
    where created>=date_sub(current_date(),{NOD})
    group by 2
    order by 1 desc
          """)
output.show(10)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 15 µs


23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:54:59 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+--------------------+
|Total_Amount|        trans_status|
+------------+--------------------+
|     1415670|OTP validation su...|
|       97310|                null|
|       30447|Challenge page ti...|
|       15349|MobileNo & EmailI...|
|       15174|Customer Cancella...|
|        6782|Customer Canellation|
|        3696|Exhausted otp att...|
|        2189|Invalid otp attem...|
|        2085|Invalid Rres message|
|        1876|Rreq not processe...|
+------------+--------------------+
only showing top 10 rows



In [86]:
NOD=90
output=spark.sql(f"""
SELECT  count(*) as Total_Amount, purchase_currency as trans_status
     FROM global_temp.data_preprocessing
    where created>=date_sub(current_date(),{NOD})
    group by 2
    order by 1 desc
          """)
output.show(10)

23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 22:58:49 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+------------+
|Total_Amount|trans_status|
+------------+------------+
|      542014|       356.0|
|       67714|       840.0|
|       28605|       784.0|
|        9185|       826.0|
|        8647|       978.0|
|        2753|       682.0|
|        2475|       980.0|
|        2146|       608.0|
|        2049|       634.0|
|        1971|        50.0|
+------------+------------+
only showing top 10 rows



In [ ]:
merchant_name

In [98]:
NOD=90
output=spark.sql(f"""
SELECT  count(*) as Total_Amount, merchant_name as trans_status
     FROM global_temp.data_preprocessing
    where created>=date_sub(current_date(),{NOD})
    group by 2
    order by 1 desc
          """)
output.show(10)

23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/10/05 23:17:54 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+--------------------+
|Total_Amount|        trans_status|
+------------+--------------------+
|       61443|               PAYTM|
|       50222|PAYTM PAYMENTS SE...|
|       22163|              ZOMATO|
|       17355|              SWIGGY|
|       16248|               Paytm|
|       15936| PASSPORTSEVAMOPSOBD|
|       13525|      ZOMATO LIMITED|
|       11947|           amazon.in|
|       11896|              PAYPAL|
|       10733| Gulf Health Counsel|
+------------+--------------------+
only showing top 10 rows



23/10/06 01:12:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 925028 ms exceeds timeout 120000 ms
23/10/06 01:12:00 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/06 01:27:59 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

23/10/06 02:01:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/10/06 02:32:37 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2088)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

...

23/10/06 09:29:11 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/10/06 09:29:21 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/10/06 09:29:31 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

23/10/06 09:29:41 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63589)
Traceback (most recent call last):
  File "/Users/nithinkumar/anaconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/nithinkumar/anaconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/nithinkumar/anaconda3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/nithinkumar/anaconda3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/nithinkumar/anaconda3/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/nithinkumar/anaconda3/lib/python3.10/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/Users/n

In [97]:
output=spark.sql(f"""
SELECT   distinct REPLACE(merchant_name, ' ', '') as merchant_names
     FROM global_temp.data_preprocessing
          """)
merchant_names=output.rdd.flatMap(lambda x :x).collect()
print((merchant_names))

['Glovo03JANKSSFAMPQC', 'LotININ', 'ESTEESMUCHOPERRO', 'mysorepalaceSBIePay', 'COBRAADVICE', 'mTaskukaardisalvestus', 'netmedsPayTM', 'CCALLIANCE', 'NovotelWarszawaCentrum', 'SPARKTELECOMLTD', '"""www.andys.md"""', 'DITTOMUSIC', 'AGODA.COMGREENVIEW', 'WWW.GTRBACKLINKS.COM', 'FlatheadDesignsRazorpay', 'BATELCO-E.SERVICES(Web)', 'IdealLinguaTranRazorpay', 'IHERBIHERB.COM0NHst', 'oplata.md*Scortel', 'alliancebroadbandPayTM', 'REDIFFCOMINDIALRazorpay', 'WeirdoughtheartRazorpay', 'LEIREGISTERINDIRazorPay', 'InnoparkTPSL', 'SCZGSMJESTAJ', 'PRIAZZAJEWELS', 'orszzSBIePay', 'SENDY.CO', 'YEREVAN-CITY.AM', 'PAYPAL*JUSTDROPPED', 'LSE-SITSApplications', 'MOMSANDHEELS.COM', 'PUPDRCARD', 'SpringteamSÃ¦by', 'MoonPay3183', 'kaizenkohinoorPayTM', 'PAYPAL*ASTRACOMMUN', 'PAYPAL*BRITISHPSYC', 'OneHairStopCashfree', 'PAYPAL*REDFAME', 'AGODA.COMBANGKOKCI', 'wwwletsvincocom', 'CHARPSTAR', 'FS*athemes', 'VideoconD2HPT', 'odishatreasurySBIePay', 'ForeverLivingProducts', 'AGODA.COMROSEPLAZA', 'OMEGA_', 'gsebserv